In [1]:
import openmm
import MDAnalysis as mda
import numpy as np
from openff.toolkit.topology.molecule import Molecule

/lila/data/chodera/lilywang/micromamba/envs/fb-193-tk-010-oe-2022/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch = "opt-geo-batch-113"
system = "19095588-8"

In [3]:
TARGET_DIRECTORY = f"../../targets/{batch}"

target_xyz = f"{TARGET_DIRECTORY}/{system}.xyz"
coordinates = mda.Universe(target_xyz).atoms.positions

target_sdf = f"{TARGET_DIRECTORY}/{system}.sdf"
mol = Molecule.from_file(target_sdf, "SDF", allow_undefined_stereo=True)
top = mol.to_topology()

In [4]:
# with open('fb-193_system.xml') as input:
#     system = openmm.XmlSerializer.deserialize(input.read())

In [5]:
integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin,
    1 / openmm.unit.picosecond,
    1 * openmm.unit.femtosecond
)
platform = openmm.Platform.getPlatformByName("Reference")

simulation = openmm.app.Simulation(
    top.to_openmm(),
    "fb-193_system.xml",
    integrator,
    platform
)
simulation.context.setPositions(coordinates * openmm.unit.angstrom)

In [6]:
simulation.minimizeEnergy(tolerance=1e-4 * openmm.unit.kilojoule_per_mole, maxIterations=10000)
for _ in range(1000):
    simulation.minimizeEnergy(tolerance=1e-4 * openmm.unit.kilojoule_per_mole, maxIterations=10)

In [7]:
state = simulation.context.getState(getPositions=True, getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(openmm.unit.kilojoule_per_mole)
opt = state.getPositions(asNumpy=True).value_in_unit(openmm.unit.angstrom)

In [8]:
opt

array([[ 2.16785134,  4.46642762,  4.35963879],
       [ 0.83968112, -1.46749801,  4.51180342],
       [ 3.52032069,  4.7170952 ,  4.65809109],
       [-0.38889787, -1.37726505,  3.82236919],
       [ 2.8407291 ,  2.67502189,  2.84000343],
       [-0.94122793,  0.65601769,  5.06150395],
       [ 1.82209311,  3.4577627 ,  3.43402202],
       [ 1.19806004, -0.45290451,  5.42747414],
       [ 0.3045394 ,  0.58324788,  5.71444389],
       [ 4.54526731,  3.96486458,  4.03918329],
       [ 4.20637472,  2.90568167,  3.14001161],
       [-1.25967571, -0.28949264,  4.06594419],
       [-1.41373635,  2.44388243,  2.2812153 ],
       [-1.75760582,  3.40487757,  3.19787358],
       [ 0.39544363,  3.22115123,  3.10311063],
       [ 5.06440332, -0.5252749 ,  6.07562458],
       [ 0.87324421,  1.52796   ,  6.73338292],
       [ 2.37926251,  1.16822147,  6.73699017],
       [ 2.47990221, -0.29921446,  6.21424193],
       [ 5.98673736,  4.37332171,  4.31888934],
       [-3.1326687 ,  3.85112088,  3.653